In [42]:
# Find all logs for ratings
allLogFiles = os.listdir('RatingResultLog/Logfiles')
cleanAllLogFiles = [i for i in allLogFiles if 'DYNRAT' in i and '_' not in i]
cleanAllLogFiles.sort()
print(len(cleanAllLogFiles))
tmp_pid = [i for i in range(1, 27) if i not in [3, 7, 8]]
pids = []
for i in tmp_pid:
    if i // 10 == 0:
        pids.append('my' + '0' + str(i))
    else:
        pids.append('my' + str(i))
print(pids)
def generate_tables(pid):
    tables = [i for i in cleanAllLogFiles if pid in i]
    # print(tables)
    rootPath = 'RatingResultLog/Logfiles'
    df1 = pd.read_csv(os.path.join(rootPath, tables[0]), sep='\t')
    df2 = pd.read_csv(os.path.join(rootPath, tables[1]), sep='\t')
    df = pd.concat([df1, df2], ignore_index=True)

    df['emotion_category'] = df['code'].apply(lambda x: 'An' if x in [10, 20] else 'Ha')
    df['emotion_type'] = df['code'].apply(lambda x: 'prototype' if x in [10, 30] else 'BO')
    
    new_order = ['code', 'emotion_category', 'emotion_type'] + [col for col in df if col not in ['code', 'emotion_category', 'emotion_type']]
    df = df[new_order]
    # print(df)
    grouped_means = df.groupby('code')[['valence', 'arousal', 'humanlike']].mean()
    # print(grouped_means)

    return df, grouped_means
    
    

ALLTables = []
AVERTables = []
for pid in pids:
    tmpALLTable, tmpAVERTable = generate_tables(pid)
    tmpALLTable['participant_id'] = pid
    tmpAVERTable['participant_id'] = pid
    ALLTables.append(tmpALLTable)
    AVERTables.append(tmpAVERTable)
# print(cleanAllLogFiles)

AVERTables = pd.concat(AVERTables)
AVERTables.reset_index(inplace=True)
AVERTables['emotion_category'] = AVERTables['code'].apply(lambda x: 'An' if x in [10, 20] else 'Ha')
AVERTables['emotion_type'] = AVERTables['code'].apply(lambda x: 'prototype' if x in [10, 30] else 'BO')
new_order = ['participant_id', 'code', 'emotion_category', 'emotion_type'] + [col for col in AVERTables if col not in ['participant_id', 'code', 'emotion_category', 'emotion_type']]
AVERTables = AVERTables[new_order]
print(AVERTables)

In [43]:
AVERTables.to_csv('mimicry_aver.csv')

In [32]:
import pandas as pd
filePath = 'RatingResultLog/Logfiles/DYNRAT1-my01.txt'

df = pd.read_csv(filePath, sep='\t')
# print(df)

In [6]:
# Group the data by 'code' and calculate the mean for each group
grouped_means = df.groupby('code')[['valence', 'arousal', 'humanlike']].mean()

print(grouped_means)

       valence   arousal  humanlike
code                               
10    3.000000  3.857143   7.285714
20    2.857143  5.142857   7.285714
30    4.571429  2.142857   4.142857
40    4.428571  1.857143   4.285714


In [8]:
import os
os.path.exists('RatingResultLog/Logfiles/DYNRAT2-my01.txt')

True

In [10]:
df2 = pd.read_csv('RatingResultLog/Logfiles/DYNRAT2-my01.txt', sep='\t')
# print(df2)
grouped_means2 = df2.groupby('code')[['valence', 'arousal', 'humanlike']].mean()
print(grouped_means2)

      valence  arousal  humanlike
code                             
10       3.25    3.750      7.125
20       3.00    5.250      6.750
30       4.75    1.750      4.750
40       3.75    2.625      4.750


In [ ]:
# concat two tables


# ANOVA analysis

In [44]:
import pandas as pd

# Load the data
data = pd.read_csv('mimicry_aver_long.csv')

# Display the first few rows to confirm it's loaded correctly
print(data.head())


   ...1 participant_id  code emotion_category emotion_type    measure  \
0     0           my01    10               An    prototype    valence   
1     0           my01    10               An    prototype    arousal   
2     0           my01    10               An    prototype  humanlike   
3     1           my01    20               An           BO    valence   
4     1           my01    20               An           BO    arousal   

      value  
0  3.133333  
1  3.800000  
2  7.200000  
3  2.933333  
4  5.200000  


In [45]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# ANOVA for each measure
for measure in data['measure'].unique():
    model = ols(f'value ~ C(emotion_category) * C(emotion_type)', data=data[data['measure'] == measure]).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA DataFrame
    print(f'ANOVA for {measure}:')
    print(anova_table)
    print('\n')


ANOVA for valence:
                                         sum_sq    df          F        PR(>F)
C(emotion_category)                   60.274831   1.0  32.924187  1.335203e-07
C(emotion_type)                        8.240048   1.0   4.500998  3.668655e-02
C(emotion_category):C(emotion_type)    3.600435   1.0   1.966681  1.643185e-01
Residual                             161.102995  88.0        NaN           NaN


ANOVA for arousal:
                                         sum_sq    df          F        PR(>F)
C(emotion_category)                   57.923913   1.0  48.454175  5.790826e-10
C(emotion_type)                        0.156957   1.0   0.131296  7.179605e-01
C(emotion_category):C(emotion_type)   17.916957   1.0  14.987788  2.074143e-04
Residual                             105.198454  88.0        NaN           NaN


ANOVA for humanlike:
                                         sum_sq    df         F    PR(>F)
C(emotion_category)                    9.352657   1.0  4.650482  0.033772

In [46]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# For simplicity, example of analyzing main effects after significant interaction
# Adjust for all combinations of categories within each measure
for measure in data['measure'].unique():
    tukey = pairwise_tukeyhsd(endog=data[data['measure'] == measure]['value'],
                              groups=data[data['measure'] == measure]['emotion_category'],
                              alpha=0.05)
    print(f'Tukey HSD results for {measure}:')
    print(tukey)


Tukey HSD results for valence:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj lower  upper  reject
-------------------------------------------------
    An     Ha   1.6188   0.0 1.0446 2.1931   True
-------------------------------------------------
Tukey HSD results for arousal:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
    An     Ha   -1.587   0.0 -2.0718 -1.1021   True
---------------------------------------------------
Tukey HSD results for humanlike:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    An     Ha  -0.6377 0.0365 -1.2345 -0.0408   True
----------------------------------------------------
